In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter_module import AnimalShelter   #Import method 

###########################
# Data Manipulation / Model
###########################
#Authentication for CRUD Module and importing
username = "aacuser2"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)  

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)
#Grazioso Salvare Logo image import
image_filename = 'GSLogo.png' # Grazioso Salvare image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode() 

#Page layout
app.layout = html.Div([
    html.Div(html.Img(src=f'data:image/png;base64,{encoded_image}')), #Display Image at top
    html.Center(html.B(html.H1('CS-340 Dashboard'))), #Display Title towards the top
    html.Center(html.B(html.H1('Elliot Huh'))), #Unique Identifier displayed under title
    html.Hr(),

    html.Div( #filtering option choices
            children=[
                dcc.RadioItems( #Radio item was utilized for filtering
                    id = 'rescue',
                    options=[ #4 options for the 4 unique rescue types
                        {'label': 'Water Rescue', 'value': 'water-rescue'},
                        {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain-rescue'},
                        {'label': 'Disaster or Individual Tracking', 'value': 'disaster-rescue'},
                        {'label': 'Reset', 'value': 'reset'},
                    ],
                    value='reset',  #Defaulted at reset option
                    labelStyle ={'display': 'inline-block', 'margin-middle':'20px'},
                ),
            ],
        style={'margin-top': '20px'}
    ),
    
    #underneath the filtering option display the datatable
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable = False,
                         filter_action = 'native', #Additional filtering
                         sort_action = 'native', #Sorting choices
                         sort_mode = "multi",
                         selected_rows = [],
                         selected_columns = [],
                         column_selectable = False,
                         row_selectable = "single", #Select to display map of animal
                         row_deletable = False,
                         page_action = "native",
                         page_current = 0,
                         page_size = 20),

    ##################
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id','data'),
              [Input('rescue', 'value')])
def update_dashboard(filter_type): #Controller for filtering options
    if filter_type == 'reset': #if reset button is selected show all/normal full data
        filtered_df = df
        
    elif filter_type == 'water-rescue': #If Water Rescue filter is chosen
        filtered_df = df[ #These are the preferred breed, genders and training ages that was associated
            (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks'] >= 26.0) & (df['age_upon_outcome_in_weeks'] <= 156.0)]
        
    elif filter_type == 'mountain-rescue': #If Mountain or Wilderness filter is chosen
        filtered_df = df[ #These are the preferred breed, genders and training ages that was associated
            (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] >= 26.0) & (df['age_upon_outcome_in_weeks'] <= 156.0)]
        
    elif filter_type == 'disaster-rescue': #If Disaster or Individual rescue filter is chosen
        filtered_df = df[ #These are the preferred breed, genders and training ages that was associated
            (df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] >= 26.0) & (df['age_upon_outcome_in_weeks'] <= 300.0)]
        
    return filtered_df.to_dict('records') #Return the updated data table

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    #Pie Chart Graph
    if viewData is None:
        return #If empty return none
    
    pie_data = pd.DataFrame.from_dict(viewData) #Utilize the data from the datatable
    #filters applied to the datatable will effect the piechart
    return [ #Return the data in utilizing breeds
        dcc.Graph( #display pie chart
            figure = px.pie(pie_data, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return None
    elif index is None:
        return None
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:12598/
